<a href="https://colab.research.google.com/github/ffsf-filho/horta_sustentavel_residencial/blob/main/horta_sustentavel_residencial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instala os SDKs necessários:
!pip install -q -U google-generativeai
!pip install gradio

In [2]:
# Importa as bibliotecas necessárias:
import google.generativeai as genai  # Biblioteca do Google para modelos de linguagem
import gradio as gr  # Biblioteca para criar interfaces web interativas
from google.colab import userdata # Biblioteca para acessar dados do usuário no Colab

# Configura a chave da API do Google:
GOOGLE_API_KEY= userdata.get('SECRET_API_GEMINI')
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
# Define oos parametros de configuração do modelo:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
}

In [4]:
# Define o modelo de linguagem a ser usado (Gemini Pro):
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

In [ ]:
# Define uma função para obter informações sobre o cultivo de uma planta:
def getInfoCultivo(plant_type):
    # Usa o modelo de linguagem para gerar texto explicando como cultivar a planta em um vaso:
    response = model.generate_content(f"explique detalhadamente como cultivar {plant_type} em um vaso, indicando qual o tamanho ideal do vaso e o tempo de colheita.")
    # Retorna o texto gerado:
    return response.text

# Define estilos CSS para a interface web:
style = """
  .gradio-container {
      background-color: #ccff99;
      color: blue;
  }
  .folhas-section {
      background-color: #bfff80;
  }
  .raizes-section {
      background-color: #ecd9c6;
  }
  .frutos-section {
      background-color: #ffffb3;
  }
  .vagens-section {
      background-color: #b3ffb3;
  }
  .ervas-section {
      background-color: #e0e0d1;
  }
  .frutas-section {
      background-color: #ffad99;
  }
"""

# Define listas de plantas para cada categoria:
folhas=['Alface', 'Rúcula', 'Espinafre', 'Couve', 'Acelga', 'Agrião']
raizes=['Cenoura', 'Rabanete', 'Beterraba']
frutos=['Tomate cereja', 'Pimentão', 'Pimenta', 'Berinjela']
vagens=['Ervilha', 'Feijão-vagem']
ervas=['Manjericão', 'salsinha', 'cebolinha', 'coentro', 'hortelã', 'orégano', 'alecrim', 'tomilho']
frutas=['Jaboticaba','Laraja anã', 'Limão anã', 'Morango', 'Tanegrina anã']

# Cria a interface web usando Gradio:
with gr.Blocks(css=style) as interface:# Add background image
    # Adiciona um título:
    gr.Markdown("# Horta em Casa: Um Guia para Cultivo em Vasos")

    # Adiciona uma breve descrição:
    gr.Markdown("## Descubra como cultivar seus próprios alimentos frescos e saudáveis em casa, mesmo com espaço limitado!")

    # Cria uma linha com duas colunas:
    with gr.Row():
        # Cria a coluna da esquerda para as categorias de plantas:
        with gr.Column(scale=1, min_width=300):
             # Cria um Accordion para as categorias de plantas:
            with gr.Accordion("Horta em Vasos", open=True):
                # Cria abas para cada categoria de planta:
                with gr.TabItem("Folhas",elem_classes='folhas-section'):
                    # Cria um botão de rádio para escolher uma folha:
                    radio_folhas = gr.Radio(folhas, label="Escolha uma folha:",elem_classes='folhas-section')
                with gr.TabItem("Raízes", elem_classes='raizes-section'):
                    radio_raizes = gr.Radio(raizes, label="Escolha uma raiz:", elem_classes='raizes-section')
                with gr.TabItem("Frutos", elem_classes='frutos-section'):
                    radio_frutos = gr.Radio(frutos, label="Escolha um fruto:", elem_classes='frutos-section')
                with gr.TabItem("Vagens", elem_classes='frutos-section'):
                    radio_vagens = gr.Radio(vagens, label="Escolha uma Vage:", elem_classes='vagens-section')
                with gr.TabItem("Ervas", elem_classes='Ervas-section'):
                    radio_ervas = gr.Radio(ervas, label="Escolha uma erva:", elem_classes='ervas-section')
                with gr.TabItem("Frutas", elem_classes='frutas-section'):
                    radio_frutas = gr.Radio(frutas, label="Escolha uma erva:", elem_classes='frutas-section')

    # Cria uma caixa de texto para exibir as informações de cultivo:
    cultivation_info = gr.Textbox(label="Forma de Cultivo", lines=5)

    # Conecta os botões de rádio à função getInfoCultivo:
    radio_folhas.change(getInfoCultivo, inputs=radio_folhas, outputs=cultivation_info)
    radio_raizes.change(getInfoCultivo, inputs=radio_raizes, outputs=cultivation_info)
    radio_frutos.change(getInfoCultivo, inputs=radio_frutos, outputs=cultivation_info)
    radio_vagens.change(getInfoCultivo, inputs=radio_vagens, outputs=cultivation_info)
    radio_ervas.change(getInfoCultivo, inputs=radio_ervas, outputs=cultivation_info)
    radio_frutas.change(getInfoCultivo, inputs=radio_frutas, outputs=cultivation_info)

# Inicia a interface web:
interface.launch(debug='true')